In [15]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from collections import Counter
from sklearn.feature_extraction.text import CountVectorizer
from nltk.corpus import stopwords
f = open('sample_patent.txt', 'r')
g = open('sample_patent2.txt', 'r')
# word_vectorizer = CountVectorizer(ngram_range = (1,2), analyzer = 'word')
# sparse_matrix = word_vectorizer.fit_transform(f)

# frequencies = sum(sparse_matrix).toarray()[0]
# pd.DataFrame(frequencies, index=word_vectorizer.get_feature_names(), columns=['frequency'])

def calc_distance(file1,file2):
    f = open(file1, 'r')
    g = open(file2, 'r')
    input_string = f.read()
    input_string2 = g.read()
    word_freqs = Counter(input_string.split())
    word_freqs2 = Counter(input_string2.split())
    for key in set(word_freqs2):
        if key not in word_freqs:
            word_freqs[key] = 0
        word_freqs[key] -= word_freqs2[key]
    distance = 0
    for val in word_freqs.values():
        distance += val**2
    distance = distance ** (.5)
    return distance

In [28]:
import MySQLdb
db = MySQLdb.connect("cal-patent-lab.chhaitskv8dz.us-west-2.rds.amazonaws.com","***REMOVED***","***REMOVED***","***REMOVED***" )
cursor = db.cursor()
cursor.execute("SELECT patent_id, invalidated from patents_decision")
data = cursor.fetchall()
count = 0
# contains invalidated/validated patent numbers
invalidated_patents = []
not_invalidated_patents = []
# contains average distances of invalidated/validated patents
invalidated_distances = []
not_invalidated_distances = []
invalidated_distances_k = []
not_invalidated_distances_k = []

for line in data:
    if line[1] == 0:
        not_invalidated_patents.append(line[0])
    elif line[1] == 1:
        invalidated_patents.append(line[0])
        

In [9]:
DB_HOST = "patentnetwork.berkeley.edu"
DB_USER = "***REMOVED***"
DB_PASSWORD = "***REMOVED***"  # Get this field from the user
DB_TABLE = "patents_decision"
DB_NAME = "***REMOVED***"
def db(host, username, psswd, database_name):
    return MySQLdb.connect(host, username, psswd, database_name)

patent_db = db(DB_HOST, DB_USER, DB_PASSWORD, DB_NAME)
cursor = patent_db.cursor()

# Get list of all patent IDs in DB
print("Getting all existing patent IDs in DB")
cursor.execute("select claims_text from patents_decision LIMIT 100000")
abstracts = cursor.fetchall()
print(abstracts[0])



Getting all existing patent IDs in DB


OperationalError: (1054, "Unknown column 'date' in 'order clause'")

('I claim: 1. 1. In combination with a height adjustable crib or the like of the type having a corner post and an extensible leg slidably connected to the corner post and having a series of holes along its length, a lock comprising PA1 A. a rigid bracket extending around the extensible leg and having sides lying flush against opposite sides of the corner post; PA1 B. means for pivotally connecting the bracket sides to the corner post so that the bracket can be swung toward and away from the leg; PA1 C. a rigid nose projecting from the bracket and extending toward the leg, said nose being arranged to engage in one of the leg holes when the bracket is swung against the leg so as to lock the leg at a selected position of lengthwise adjustment relative to the corner post, and PA1 D. means mounted on the corner post and cooperating with the bracket for removably retaining the bracket with its nose in said hole said retaining means comprising 1. a latch pivotally connected to the corner post

In [16]:

stopWords = stopwords.words('english')

# count_vect = CountVectorizer(stop_words  = stopWords)
# content = ["How to format my hard disk hello", " Hard disk format problems "]
# X = count_vect.fit_transform(content)
# print count_vect.get_feature_names()
# print count_vect.fit_transform(content).toarray()
# print count_vect.vocabulary


In [18]:
from os import listdir
from os.path import isfile, join
files = [f for f in listdir("patents") if isfile(join("patents", f))]
patent_numbers = [name[:len(name)-4] for name in files]
#patent_numbers



In [19]:
test_files = files[1:]

input_file = "patents/"+files[0]
abs_list = []
for name in test_files:
    f = open("patents/" + name, 'r')
    abs_list.append(f.read())

In [20]:
print len(abs_list)
print len(test_files)

5899
5899


In [29]:
count_vect = CountVectorizer(stop_words  = stopWords)
abs_lst = [i[0] for i in abstracts ][:5000]
abs_lst.extend(invalidated_patents)
abs_lst.extend(not_invalidated_patents)
X = count_vect.fit_transform(abs_lst)
print count_vect.fit_transform(abs_lst).toarray().shape


(6654, 26708)


In [45]:
import numpy as np
invalidated_distances_k = []
not_invalidated_distances_k = []
inval_num = len(invalidated_patents)
not_inval_num = len(not_invalidated_patents)
for i in range(inval_num):
    dist = (X.toarray()[:] - X[5000+i])
    dist = np.square(dist)
    dist = np.sum(dist, axis = 1)
    dist = np.sqrt(dist)
#     dist.remove(min(dist))
    dist = np.average(dist)
    invalidated_distances_k.append(dist)
    print(i)
for i in range(not_inval_num):
    dist = (X.toarray()[:] - X[5000+inval_num + i])
    dist = np.square(dist)
    dist = np.sum(dist, axis = 1)
    dist = np.sqrt(dist)
#     dist.remove(min(dist))
    dist = np.average(dist)
    not_invalidated_distances_k.append(dist)
    print(i)
# single = X.toarray()[0]
# dist = (X.toarray()[1:]-single) **2
# dist = np.sum(dist, axis = 1)
# dist = np.sqrt(dist)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253


KeyboardInterrupt: 

In [55]:
print(len(invalidated_distances), len(not_invalidated_distances))
print(len(invalidated_distances_k), len(not_invalidated_distances_k))
print(sum(invalidated_distances)/float(len(invalidated_distances)))
print(sum(not_invalidated_distances)/float(len(not_invalidated_distances)))
print(sum(invalidated_distances_k)/float(len(invalidated_distances_k)))
print(sum(not_invalidated_distances_k)/float(len(not_invalidated_distances_k)))


(16, 18)
(16, 18)
22.4712228211
23.2831912955
15.281794673
16.0324290254


In [20]:
import matplotlib.pyplot as plt
import numpy as np

# import plotly.plotly as py
# Learn about API authentication here: https://plot.ly/python/getting-started
# Find your api_key here: https://plot.ly/settings/api

plt.hist(invalidated_distances, bins=range(int(min(invalidated_distances)),1))
plt.title("'Invalidated'Distance Histogram. Average: 22.4")
plt.xlabel("Distance")
plt.ylabel("Frequency")

fig = plt.gcf()
axes = plt.gca()
axes.set_ylim([0,8])

plt.figure()
plt.hist(not_invalidated_distances, bins=range(int(min(not_invalidated_distances)) 1))
plt.title("'Not Invalidated' Distance Histogram. Average: 23.2")
plt.xlabel("Distance")
plt.ylabel("Frequency")

axes = plt.gca()
axes.set_ylim([0,8])
plt.show()
# plot_url = py.plot_mpl(fig, filename='mpl-basic-histogram')


In [13]:
import pickle
with open('distances.pickle', 'w') as f:  # Python 3: open(..., 'wb')
    pickle.dump([distances,not_invalidated_distances,invalidated_distances,not_invalidated_distances_k,invalidated_distances_k], f)

In [10]:
len(invalidated)
# len(not_invalidated)

777

In [10]:

len(test_files)

77831

In [14]:
import pickle
with open('distances.pickle', 'r') as f:  # Python 3: open(..., 'wb')
    [distances,not_invalidated_distances,invalidated_distances,not_invalidated_distances_k,invalidated_distances_k] = pickle.load(f)

In [18]:
import matplotlib.pyplot as plt
import numpy as np
data = np.vstack([invalidated_distances, not_invalidated_distances]).T
plt.hist((invalidated_distances, not_invalidated_distances), bins=range(int(min(invalidated_distances)),30, 1))
plt.title("'Invalidated'Distance Histogram. Average: 22.4")
plt.xlabel("Distance")
plt.ylabel("Frequency")
plt.show()

ValueError: all the input array dimensions except for the concatenation axis must match exactly